# Introduction to Coroutines in Python

Sources:
* http://www.informit.com/articles/article.aspx?p=2320938
* http://stackoverflow.com/questions/9708902/in-practice-what-are-the-main-uses-for-the-new-yield-from-syntax-in-python-3/26109157#26109157

## What is a Coroutine?

A coroutine is a generalization of a subroutine (or method/function). Contrary to a subroutine, a coroutine can be *suspended* and *resumed* at certain locations.

## An Introductory Example

In [2]:
import asyncio

For documentation purposes, all coroutines should be decorated with ``@asyncio.coroutine``, but this cannot be strictly enforced.

Here's a simple coroutine that counts (prints) to $10$:

In [3]:
@asyncio.coroutine
def counter():
    for i in range(10):
        print(i)
        yield

In [4]:
coro = counter()

In [5]:
next(coro)
next(coro)
next(coro)

0
1
2


Coroutines can also "return" and retrieve values:

In [6]:
@asyncio.coroutine
def counter():
    for i in range(10):
        x = yield i
        print("x: {}".format(x))

In [7]:
coro = counter()

In [8]:
next(coro)

0

In [9]:
next(coro)

x: None


1

In [10]:
coro.send("bla")

x: bla


2

In [11]:
next(coro)

x: None


3

blaa

In [12]:
@asyncio.coroutine
def match(pattern):
    print("Looking for: " + pattern)
    try:
        while True:
            s = yield
            if pattern in s:
                print("found {} in {}".format(pattern, s))
    except GeneratorExit:
        print("Done")

In [13]:
m = match("gugus")
m.__next__()  # executes function body until "yield" statement

Looking for: gugus


In [14]:
m.send("blagugus")  # evaluation continues inside match() until it encounters "yield"

found gugus in blagugus


In [15]:
m.send("asdfsadf")

In [16]:
m.send("gugusasdflj")

found gugus in gugusasdflj


In [17]:
m.close()

Done


We can chain functions that ``send()`` and functions that ``yield`` together to obtain complex behaviors.

In [18]:
@asyncio.coroutine
def read(text, coroutine):
    for word in text.split():
        coroutine.send(word)
    coroutine.close()

In [19]:
text = "Commending spending is offending to people pending lending!"
matcher = match("ending")
matcher.__next__()

Looking for: ending


In [20]:
read(text, matcher)

<generator object coro at 0x7f58f41589d8>

Note that ``match()`` is a "consumer" of "filter" (``yield``s, but does not ``send()``), while ``read()`` is a "producer" (produces data and sends it to another coroutine).
A producer or filter can of course have multiple coroutines downstream of it. Here is an example:

In [21]:
@asyncio.coroutine
def read_to_many(text, coroutines):
    for word in text.split():
        for c in coroutines:
            c.send(word)
    for c in coroutines:
        c.close()

In [22]:
m1 = match("mend")
m1.__next__()

Looking for: mend


In [23]:
m2 = match("pe")
m2.__next__()

Looking for: pe


In [24]:
read_to_many(text, [m1, m2])

<generator object coro at 0x7f58f4158b88>

## The ``yield from`` syntax

Note that ``yield from g`` is **not** equivalent to ``for v in g: yield v``. Think of ``yield from`` as providing a transparent two-way channel from the caller to the sub-coroutine. That includes both getting data from and sending data to the sub-coroutine.

Consider the following example:

In [25]:
@asyncio.coroutine
def reader():
    """A generator that fakes a read from a file, socket, etc."""
    for i in range(4):
        yield '<< {}'.format(i)

@asyncio.coroutine
def reader_wrapper(coro):
    """Manually iterate over data produced by reader"""
    for v in coro:
        yield v

wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

<< 0
<< 1
<< 2
<< 3


We can simplify ``reader_wrapper`` as follows:

In [26]:
@asyncio.coroutine
def reader_wrapper(coro):
    yield from coro
    
wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

<< 0
<< 1
<< 2
<< 3


Now let's create a coroutine that writes data to some file descriptor:

In [27]:
@asyncio.coroutine
def writer():
    """A coroutine that writes data *sent* to it to fd, socket, etc."""
    while True:
        w = yield
        print('>> ', w)

A coroutine wrapping ``writer`` could look as follows:

In [40]:
@asyncio.coroutine
def writer_wrapper(coro):
    coro.send(None)  # prime the coroutine (run to 1st yield statement)
    while True:
        try:
            x = yield  # Capture the value that's sent
            coro.send(x)  # and pass it to the writer
        except StopIteration:
            pass

In [41]:
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)  # "prime" the coroutine
for i in range(4):
    wrap.send(i)

>>  0
>>  1
>>  2
>>  3


However, we can simplify ``writer_wrapper`` as follows:

In [42]:
@asyncio.coroutine
def writer_wrapper(coro):
    yield from coro
    
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)  # "prime" the coroutine
for i in range(4):
    wrap.send(i)

>>  0
>>  1
>>  2
>>  3


Let's make it more complicated. What if our writer needs to handle exceptions? Let's say the writer handles a ``SpamException`` and it prints ``***`` if it encounters one.